In [ ]:
%matplotlib notebook
#from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
#from matplotlib import cm

from functools import partial

import numpy as np
from scipy.optimize import minimize

In [ ]:
%run ../scripts/OS2015_academic_problem.py

config = {'num_coarse_grid_elements': [4, 4], 
          'num_grid_refinements': 2,
          'num_grid_subdomains': [2, 2], 
          'num_grid_oversampling_layers': 4, # num_grid_oversampling_layers has to exactly cover one subdomain!
          'initial_RB_order': 0,
          'enrichment_target_error': 2.,
          'marking_doerfler_theta': 0.33,
          'marking_max_age': 2}

grid_and_problem_data = init_grid_and_problem(config)

mu_bar = grid_and_problem_data['mu_bar']
parameter_range = grid_and_problem_data['parameter_range']
parameter_range = (parameter_range[0], parameter_range[1])

initial_guess = 0.5*(parameter_range[0] + parameter_range[1])

In [ ]:
%run ../scripts/discretize_elliptic.py

d_blocked, block_space, local_boundary_info = discretize(grid_and_problem_data)
d_blocked.disable_logging()
mu_bar = d_blocked.parse_parameter(mu_bar)

def detailed_quantity_of_interest(mu):
    return d_blocked.rhs.apply(d_blocked.solve(mu)).data[0]

print('computing some detailed quantities of interest:')

training_set = d_blocked.parameter_space.sample_uniformly(5)
training_set.extend(d_blocked.parameter_space.sample_randomly(10))
training_set = [mu['diffusion'] for mu in training_set]
training_set.sort()
detailed_quantities_of_interest = [detailed_quantity_of_interest(mu) for mu in training_set]

plt.plot(training_set, detailed_quantities_of_interest, '-o')

In [ ]:
print('minimizing detailed quantity of interest ', end='')

detailed_minimization_data = {'num_evals': 0,
                              'all_evaluation_points': [],
                              'evaluation_points': [initial_guess, ]}


def quantity_of_interest(function, cfg, mu):
    print('.', end='')
    cfg['num_evals'] += 1
    cfg['all_evaluation_points'].append(mu)
    return function(mu)


def postprocess(result, cfg, evaluate_qoi):
    if (result.status != 0):
        print(' failed!')
    else:
        print(' succeded!')
        print('  x_min:    {}'.format(result.x))
        print('  qoi(x_min): {}'.format(result.fun))
        print('  num iterations:     {}'.format(result.nit))
        print('  num function calls: {}'.format(cfg['num_evals']))

        plt.plot(training_set, detailed_quantities_of_interest)
        plt.plot(cfg['all_evaluation_points'], [evaluate_qoi(mu) for mu in cfg['all_evaluation_points']], 'o')

result = minimize(partial(quantity_of_interest, detailed_quantity_of_interest, detailed_minimization_data),
                  initial_guess,
                  method='L-BFGS-B', jac=False,
                  bounds=(parameter_range,),
                  callback=lambda xk: detailed_minimization_data['evaluation_points'].append(xk),
                  options={'ftol': 1e-15, 'gtol': 1e-15})

postprocess(result, detailed_minimization_data, detailed_quantity_of_interest)

In [ ]:
print('reducing with standard RB:', flush=True)

from pymor.algorithms.greedy import greedy
from pymor.discretizations.basic import StationaryDiscretization
from pymor.parameters.functionals import GenericParameterFunctional
from pymor.reductors.coercive import CoerciveRBReductor

d_unblocked = StationaryDiscretization(
    d_blocked.operators['global_op'], d_blocked.operators['global_rhs'],
    products={'energy_dg_mu_bar': d_blocked.operators['global_op'].assemble(mu_bar)},
    parameter_space=d_blocked.parameter_space,
    name='unblocked_swipdg')

def coercivity_estimator(mu):
    return 1./np.sqrt(alpha(grid_and_problem_data['lambda']['coefficients'], mu, mu_bar))

reductor_unblocked = CoerciveRBReductor(
    d_unblocked,
    product=d_unblocked.energy_dg_mu_bar_product,
    coercivity_estimator=GenericParameterFunctional(coercivity_estimator,
                                                    grid_and_problem_data['parameter_type']))

greedy_data = greedy(d_unblocked, reductor_unblocked, training_set,
                     extension_params={'method': 'gram_schmidt'},
                     max_extensions=9999)

print('took {} evaluations of the detailed discretization'.format(greedy_data['extensions']))
print('maximum estimated model reduction error over training set: {}'.format(np.min(greedy_data['max_errs'])))

In [ ]:
rd_unblocked = greedy_data['reduced_discretization']
rd_unblocked.disable_logging()

def reduced_quantity_of_interest(mu):
    return rd_unblocked.rhs.apply(rd_unblocked.solve(mu)).data[0]

print('computing some reduced quantities of interest ...')

reduced_quantities_of_interest = [reduced_quantity_of_interest(mu) for mu in training_set]

print('L-infty error of QoI: {}'.format(np.max(np.abs(  np.array(detailed_quantities_of_interest)
                                                      - np.array(reduced_quantities_of_interest)))))

plt.plot(training_set, detailed_quantities_of_interest)
plt.plot(training_set, reduced_quantities_of_interest, 'o')

In [ ]:
print('minimizing reduced quantity of interest ', end='')

reduced_minimization_data = {'num_evals': 0,
                             'all_evaluation_points': [],
                             'evaluation_points': [initial_guess, ]}

result = minimize(partial(quantity_of_interest, reduced_quantity_of_interest, reduced_minimization_data),
                  initial_guess,
                  method='L-BFGS-B', jac=False,
                  bounds=(parameter_range,),
                  callback=lambda xk: reduced_minimization_data['evaluation_points'].append(xk),
                  options={'ftol': 1e-15, 'gtol': 1e-15})

postprocess(result, reduced_minimization_data, reduced_quantity_of_interest)
plt.plot(detailed_minimization_data['all_evaluation_points'],
         [detailed_quantity_of_interest(mu) for mu in detailed_minimization_data['all_evaluation_points']], 'x')

In [ ]:
# this is required to estimate the error tolerance for the online enrichment
print('estimating some detailed errors:')
detailed_errors = []                                                                                                                                                                                                      
for mu in parameter_range:
    mu = d_unblocked.parse_parameter(mu)
    print('  {}: '.format(mu), end='', flush=True)
    U = d_unblocked.solve(mu)
    estimate = d_unblocked.estimate(U, mu=mu)
    print(estimate)
    detailed_errors.append(estimate)

In [ ]:
from pymor.core.exceptions import ExtensionError

max_extension_error = 1.
print('maximum extension error: {}'.format(max_extension_error))
print('')

%run ../scripts/offline.py

reductor_blocked = init_local_reduced_bases(d_blocked, block_space, config['initial_RB_order'])

#print('adding some global solution snapshots to reduced basis ...', flush=True)
#for mu in parameter_range:
#    U = d.solve(mu)
#    try:
#        reductor_blocked.extend_basis(U)
#    except ExtensionError:
#        pass                                                                                                                                                                                                     
#print('')

print('reducing:', flush=True)
rd_blocked = reductor_blocked.reduce()
rd_blocked = rd_blocked.with_(estimator=d_blocked.estimator)
rd_blocked.disable_logging()
print('initial reduced (LRBMS) system is of size {}x{}'.format(rd_blocked.solution_space.dim, rd_blocked.solution_space.dim))

In [ ]:
def LRBMS_quantity_of_interest(mu):
    return rd_blocked.rhs.apply(rd_blocked.solve(mu)).data[0]

print('computing some (LRBMS) reduced quantities of interest ...')

LRBMS_quantities_of_interest = [LRBMS_quantity_of_interest(mu) for mu in training_set]

print('L-infty error w.r.t. detailed QoI: {}'.format(np.max(np.abs(  np.array(detailed_quantities_of_interest)
                                                        - np.array(LRBMS_quantities_of_interest)))))
print('L-infty error w.r.t reduced QoI: {}'.format(np.max(np.abs(  np.array(reduced_quantities_of_interest)
                                                        - np.array(LRBMS_quantities_of_interest)))))

plt.plot(training_set, detailed_quantities_of_interest)
plt.plot(training_set, LRBMS_quantities_of_interest, 'o')

In [ ]:
print('minimizing reduced (LRBMS) quantity of interest ', end='')

LRBMS_minimization_data = {'num_evals': 0,
                           'all_evaluation_points': [],
                           'evaluation_points': [initial_guess, ]}

result = minimize(partial(quantity_of_interest, LRBMS_quantity_of_interest, LRBMS_minimization_data),
                  initial_guess,
                  method='L-BFGS-B', jac=False,
                  bounds=(parameter_range,),
                  callback=lambda xk: LRBMS_minimization_data['evaluation_points'].append(xk),
                  options={'ftol': 1e-15, 'gtol': 1e-15})

print(result)
#postprocess(result, LRBMS_minimization_data, LRBMS_quantities_of_interest)
#plt.plot(detailed_minimization_data['all_evaluation_points'],
#         [LRBMS_quantities_of_interest(mu) for mu in LRBMS_minimization_data['all_evaluation_points']], 'x')